In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install webdriver-manager # Install the missing 'webdriver_manager' module

In [ ]:
!pip install --upgrade webdriver-manager

In [ ]:
#Import necessary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time


In [32]:

# Function to convert currency to INR
def convert_to_inr(price, currency):
    conversion_rates = {
        'USD': 83.12,
        'EUR': 90.73,
        'CHF': 95.73,
        'KWD': 270.04,
        'AED': 22.63
    }
    return round(price * conversion_rates.get(currency, 1))

# Setup Selenium WebDriver
options = Options()
options.headless = False  # Change to True to run in headless mode

# driver = webdriver.Chrome(service=Service(ChromeDriverManager(version='126.0.6478.63').install()))
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



# Function to scrape data from Zillow
def scrape_zillow():
    url = "https://www.zillow.com/"
    driver.get(url)

    # Click on the element with the specified class
    element_class = "Anchor-c11n-8-100-6__sc-hn4bge-0 pfs__sc-1dpbk03-0 hfUBNo dGfGKQ noroute"
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, element_class))
        )
        ActionChains(driver).move_to_element(element).click().perform()
    except Exception as e:
        print(f"Error clicking on the element: {e}")
        return []

    time.sleep(5)  # Wait for the page to load (adjust this if necessary)

    # Extract property data
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    properties = soup.find_all('div', class_='StyledPropertyCardDataWrapper-c11n-8-101-0__sc-hfbvv9-0 hHVup property-card-data')

    data = []
    for property in properties:
        try:
            price = property.find('span', class_='price').text if property.find('span', class_='price') else None
            address = property.find('address', class_='address').text if property.find('address', class_='address') else None
            city = address.split(",")[1].strip() if address else None
            details = property.find_all('span', class_='detail')
            area = details[0].text if len(details) > 0 else None
            bedrooms = details[1].text if len(details) > 1 else None
            bathrooms = details[2].text if len(details) > 2 else None

            if price:
                price = int(price.replace('$', '').replace(',', ''))
                price_inr = convert_to_inr(price, 'USD')
            else:
                price_inr = None

            data.append({
                'address': address,
                'city': city,
                'price_inr': price_inr,
                'area': area,
                'bedrooms': bedrooms,
                'bathrooms': bathrooms
            })
        except Exception as e:
            print(f"Error parsing property: {e}")

    return data

# Scrape multiple websites and aggregate data
def aggregate_data():
    data = []
    data.extend(scrape_zillow())
    # Add functions for other websites...

    # Ensure minimum entries per city and total entries
    df = pd.DataFrame(data)
    city_counts = df['city'].value_counts()
    cities_to_keep = city_counts[city_counts >= 100].index
    df = df[df['city'].isin(cities_to_keep)]

    if len(df) < 3000:
        print("Not enough data. Please scrape more properties.")
    else:
        df.to_csv('properties_data.csv', index=False)

aggregate_data()
driver.quit()


AttributeError: 'NoneType' object has no attribute 'split'

In [31]:
# # Import necessary libraries
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.action_chains import ActionChains
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# # Function to convert currency to INR
# def convert_to_inr(price, currency):
#     conversion_rates = {
#         'USD': 83.12,
#         'EUR': 90.73,
#         'CHF': 95.73,
#         'KWD': 270.04,
#         'AED': 22.63
#     }
#     return round(price * conversion_rates.get(currency, 1))

# # Setup Selenium WebDriver
# options = Options()
# options.headless = False  # Change to True to run in headless mode

# # Initialize ChromeDriverManager
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# # Function to scrape data from Zillow
# def scrape_zillow(driver):
#     url = "https://www.zillow.com/"
#     driver.get(url)

#     # Click on the element with the specified class
#     element_class = "Anchor-c11n-8-100-6__sc-hn4bge-0 pfs__sc-1dpbk03-0 hfUBNo dGfGKQ noroute"
#     try:
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CLASS_NAME, element_class))
#         )
#         ActionChains(driver).move_to_element(element).click().perform()
#     except Exception as e:
#         print(f"Error clicking on the element: {e}")
#         return []

#     time.sleep(5)  # Wait for the page to load (adjust this if necessary)

#     # Extract property data
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     properties = soup.find_all('div', class_='StyledPropertyCardDataWrapper-c11n-8-101-0__sc-hfbvv9-0 hHVup property-card-data')

#     data = []
#     for property in properties:
#         try:
#             price = property.find('span', class_='price').text if property.find('span', class_='price') else None
#             address = property.find('address', class_='address').text if property.find('address', class_='address') else None
#             city = address.split(",")[1].strip() if address and "," in address else None
#             details = property.find_all('span', class_='detail')
#             area = details[0].text if len(details) > 0 else None
#             bedrooms = details[1].text if len(details) > 1 else None
#             bathrooms = details[2].text if len(details) > 2 else None

#             if price:
#                 price = int(price.replace('$', '').replace(',', ''))
#                 price_inr = convert_to_inr(price, 'USD')
#             else:
#                 price_inr = None

#             data.append({
#                 'address': address,
#                 'city': city,
#                 'price_inr': price_inr,
#                 'area': area,
#                 'bedrooms': bedrooms,
#                 'bathrooms': bathrooms
#             })
#         except Exception as e:
#             print(f"Error parsing property: {e}")

#     return data

# # Scrape multiple websites and aggregate data
# def aggregate_data():
#     data = []
#     data.extend(scrape_zillow(driver))
#     # Add functions for other websites...

#     # Ensure minimum entries per city and total entries
#     df = pd.DataFrame(data)
#     if 'city' in df.columns:
#         city_counts = df['city'].value_counts()
#         cities_to_keep = city_counts[city_counts >= 100].index
#         df = df[df['city'].isin(cities_to_keep)]

#         if len(df) < 3000:
#             print("Not enough data. Please scrape more properties.")
#         else:
#             df.to_csv('properties_data.csv', index=False)
#     else:
#         print("No valid data scraped.")

# aggregate_data()
# driver.quit()


In [ ]:
pip install webdriver-manager


In [30]:
# # Import necessary libraries
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.action_chains import ActionChains
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# # Function to convert currency to INR
# def convert_to_inr(price, currency):
#     conversion_rates = {
#         'USD': 83.12,
#         'EUR': 90.73,
#         'CHF': 95.73,
#         'KWD': 270.04,
#         'AED': 22.63
#     }
#     return round(price * conversion_rates.get(currency, 1))

# # Setup Selenium WebDriver
# options = Options()
# options.headless = False  # Change to True to run in headless mode

# # Initialize ChromeDriverManager
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

# # Function to scrape data from Zillow
# def scrape_zillow(driver):
#     url = "https://www.zillow.com/"
#     driver.get(url)

#     # Click on the element with the specified class
#     element_class = "Anchor-c11n-8-100-6__sc-hn4bge-0 pfs__sc-1dpbk03-0 hfUBNo dGfGKQ noroute"
#     try:
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CLASS_NAME, element_class))
#         )
#         ActionChains(driver).move_to_element(element).click().perform()
#     except Exception as e:
#         print(f"Error clicking on the element: {e}")
#         return []

#     time.sleep(5)  # Wait for the page to load (adjust this if necessary)

#     # Extract property data
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     properties = soup.find_all('div', class_='StyledPropertyCardDataWrapper-c11n-8-101-0__sc-hfbvv9-0 hHVup property-card-data')

#     data = []
#     for property in properties:
#         try:
#             price = property.find('span', class_='price').text if property.find('span', class_='price') else None
#             address = property.find('address', class_='address').text if property.find('address', class_='address') else None
#             city = address.split(",")[1].strip() if address and "," in address else None
#             details = property.find_all('span', class_='detail')
#             area = details[0].text if len(details) > 0 else None
#             bedrooms = details[1].text if len(details) > 1 else None
#             bathrooms = details[2].text if len(details) > 2 else None

#             if price:
#                 price = int(price.replace('$', '').replace(',', ''))
#                 price_inr = convert_to_inr(price, 'USD')
#             else:
#                 price_inr = None

#             data.append({
#                 'address': address,
#                 'city': city,
#                 'price_inr': price_inr,
#                 'area': area,
#                 'bedrooms': bedrooms,
#                 'bathrooms': bathrooms
#             })
#         except Exception as e:
#             print(f"Error parsing property: {e}")

#     return data

# # Scrape multiple websites and aggregate data
# def aggregate_data():
#     data = []
#     data.extend(scrape_zillow(driver))
#     # Add functions for other websites...

#     # Ensure minimum entries per city and total entries
#     df = pd.DataFrame(data)
#     if 'city' in df.columns:
#         city_counts = df['city'].value_counts()
#         cities_to_keep = city_counts[city_counts >= 100].index
#         df = df[df['city'].isin(cities_to_keep)]

#         if len(df) < 3000:
#             print("Not enough data. Please scrape more properties.")
#         else:
#             df.to_csv('properties_data.csv', index=False)
#     else:
#         print("No valid data scraped.")

# aggregate_data()
# driver.quit()
